In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

### To get started first we'll look at the GDP data of US yearwise.
So that we could see when the recession started, when it ended and what's the recession bottom.

In [2]:
gdp = pd.read_excel('gdplev.xls')
gdp.head(15)

,"Current-Dollar and ""Real"" Gross Domestic Product",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,2016-09-29 00:00:00
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Annual,NaN,NaN,NaN,Quarterly,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,(Seasonally adjusted annual rates),NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,GDP in billions of current dollars,GDP in billions of chained 2009 dollars,NaN,NaN,GDP in billions of current dollars,GDP in billions of chained 2009 dollars,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1929,104.6,1056.6,NaN,1947q1,243.1,1934.5,NaN
8,1930,92.2,966.7,NaN,1947q2,246.3,1932.3,NaN
9,1931,77.4,904.8,NaN,1947q3,250.1,1930.3,NaN


We only need columns 4,6. Because we need quaterly data.
<br>
Since we need the data starting from first quarter of 2000. We need to find the number of rows to skip.

In [3]:
gdp = gdp.iloc[:,[4,6]]
gdp.columns = ['Year','GDP']
x = gdp.loc[gdp['Year']=='2000q1'].index[0]
x = [a for a in range(x)]
gdp = gdp.drop(x,axis=0)
gdp = gdp.reset_index(drop=True)
gdp

,Year,GDP
0,2000q1,12359.1
1,2000q2,12592.5
2,2000q3,12607.7
3,2000q4,12679.3
4,2001q1,12643.3
...,...,...
61,2015q2,16374.2
62,2015q3,16454.9
63,2015q4,16490.7
64,2016q1,16525


Now to get the recession start, end and the recession bottom year. We'll use pandas shift function.
<br>
The idea is pretty simple first we'll look for the gdp decline in one consecutive quarter and then check whether it declined again in the next quarter.

In [4]:
x = gdp['GDP']<gdp['GDP'].shift()
start = 0
for i in range(len(x)):
    if x[i]==x[i+1]==True:
        start = i
        break
    else: continue
end = 0
for i in range(start,len(x)):
    if x[i]==x[i+1]==False:
        end = i+1
        break
    else: continue
bottom = (gdp.loc[start:end,'GDP'].astype(float)).idxmin()
startyr = gdp['Year'][start]
endyr = gdp['Year'][end]
bottomyr = gdp['Year'][bottom]
print('The recession starts in {} and ends in {} with the reccesion bottom in {}'.format(startyr,endyr,bottomyr))

The recession starts in 2008q3 and ends in 2009q4 with the reccesion bottom in 2009q2


### Now we'll look at the mean housing price data

In [5]:
df = pd.read_csv('City_Zhvi_AllHomes.csv')
df

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,17426,Chicago,IL,Chicago,Cook,3,109700.0,109400.0,109300.0,109300.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,13271,Philadelphia,PA,Philadelphia,Philadelphia,4,50000.0,49900.0,49600.0,49400.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,40326,Phoenix,AZ,Phoenix,Maricopa,5,87200.0,87700.0,88200.0,88400.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10725,398292,Town of Wrightstown,WI,Green Bay,Brown,10726,NaN,NaN,NaN,NaN,...,149900,150100,150300,150000,149200,149900,151400,152500,154100,155900
10726,398343,Urbana,NY,Corning,Steuben,10727,66900.0,65800.0,65500.0,65100.0,...,135700,136400,137700,138700,140500,143600,145000,144000,143000,143000
10727,398496,New Denmark,WI,Green Bay,Brown,10728,NaN,NaN,NaN,NaN,...,188700,189800,190800,191200,191200,191700,192800,194000,196300,198900
10728,398839,Angels,CA,NaN,Calaveras,10729,115600.0,116400.0,118000.0,119000.0,...,280400,279600,278000,276600,275000,273700,272000,269100,269000,270900


In this dataframe, we don't have state names, we have their shortforms so we need state names and their fullforms.

In [6]:
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

Ok so now we'll extract State and RegionName and the pricing data from the year 2000 onwards
<br>
To do so, we first create a new dataframe with State and RegionName and a seperate dataframe for housing prices and later merge them together.
<br>
We're doing this because we can see that the prices are monthly but we need quarterly so we have to do certain operations to clean the housing data.

In [7]:
y = df.iloc[:,1:3]
y = y.set_index('State')
y['S'] = pd.Series(states)
y = y.reset_index()
y = y[['RegionName','S']]
y.columns = ['RegionName','State']
y

,RegionName,State
0,New York,New York
1,Los Angeles,California
2,Chicago,Illinois
3,Philadelphia,Pennsylvania
4,Phoenix,Arizona
...,...,...
10725,Town of Wrightstown,Wisconsin
10726,Urbana,New York
10727,New Denmark,Wisconsin
10728,Angels,California


Now in order to clean it we will resample the data quarterly and then return the mean housing value of each quarter.

In [8]:
ind = df.columns.get_loc("2000-01")
x = df.iloc[:,ind:]
lst = [pd.Period(i) for i in list(x.columns)]
x.columns = lst
x = x.resample('Q',axis=1).mean()
x

,2000Q1,2000Q2,2000Q3,2000Q4,2001Q1,2001Q2,2001Q3,2001Q4,2002Q1,2002Q2,...,2014Q2,2014Q3,2014Q4,2015Q1,2015Q2,2015Q3,2015Q4,2016Q1,2016Q2,2016Q3
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
1,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
2,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
3,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
4,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10725,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
10726,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
10727,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0
10728,151000.000000,155900.000000,158100.000000,167466.666667,176833.333333,183766.666667,190233.333333,184566.666667,184033.333333,186133.333333,...,244466.666667,254066.666667,259933.333333,260100.000000,250633.333333,263500.000000,279500.000000,276533.333333,271600.000000,269950.0


In [9]:
mhp = y.merge(x,left_index=True,right_index=True)
mhp = mhp.set_index(['State','RegionName'])
mhp.columns = [str(i).lower() for i in mhp.columns]
mhp

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
New York,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
Wisconsin,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


### And at last we need the university towns

In [10]:
un = pd.read_csv('university_towns.txt',sep='/n',header=None,engine='python')
un

,0
0,Alabama[edit]
1,Auburn (Auburn University)[1]
2,Florence (University of North Alabama)
3,Jacksonville (Jacksonville State University)[2]
4,Livingston (University of West Alabama)[2]
...,...
562,Stevens Point (University of Wisconsinâ€“Steve...
563,Waukesha (Carroll University)
564,Whitewater (University of Wisconsinâ€“Whitewat...
565,Wyoming[edit]


Alright so we need the State names as well as Region names of the States. In order to do that, first we'll clean the data and then seperate them in States and Regions.
<br>
Rows containing **[edit]** keyword contains the State names and all the rows following the State contains the Regions of a State.
<br>
So we'll remove everything except State and Region Name.

In [11]:
un[0] = un[0].replace(' \(.*','',regex=True)
un[0] = un[0].replace('â€“',' ',regex=True)

Now we'll extract the indices of the rows with State names and create a new column named 'State' in our dataset.

In [12]:
univ = un.copy()
x = un[0].str.contains('\[edit\]')
univ['State']=univ[x]
a = 0
for i in x:
    if i == True:
        b = univ[0][a]
    elif i==False:
        univ['State'][a] = b
    a+=1

Now we'll extract the indices of rows which contains State names in initial column. Then we'll remove the [edit] keyword from them. Then we'll create a final dataset with 'State' and 'RegionName' as column labels.

In [13]:
ind = list(un[x].index)
univ = univ.drop(labels = ind)
univ['State'] = univ['State'].replace('\[edit\]','',regex=True)
univ.columns = ['RegionName','State']
univ = pd.DataFrame(list(univ['RegionName']),list(univ['State']))
univ = univ.reset_index()
univ.columns = ['State','RegionName']
univ

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
512,Wisconsin,River Falls
513,Wisconsin,Stevens Point
514,Wisconsin,Waukesha
515,Wisconsin,Whitewater


## Hypothesis Testing
Now to test our hypothesis we'll run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)
To do so we'll extract the mean housing price data for university towns and remove those towns from the mhp data so that we can have two seperate datasets, one for university towns and one for non university towns.

In [14]:
univ = univ.set_index(['State','RegionName'])
mhp = mhp.loc[:,[startyr,bottomyr]]
mhp['Price Ratio'] = mhp[startyr]/mhp[bottomyr]
univ = univ.merge(mhp,how='inner',left_index=True,right_index=True)
mhp = mhp[~mhp.index.isin(univ.index)]
mhp

,,2008q3,2009q2,Price Ratio
State,RegionName,,,
New York,New York,499766.666667,465833.333333,1.072844
California,Los Angeles,469500.000000,413900.000000,1.134332
Illinois,Chicago,232000.000000,219700.000000,1.055985
Pennsylvania,Philadelphia,116933.333333,116166.666667,1.006600
Arizona,Phoenix,193766.666667,168233.333333,1.151773
...,...,...,...,...
Wisconsin,Town of Wrightstown,149633.333333,144833.333333,1.033142
New York,Urbana,117900.000000,123433.333333,0.955171
Wisconsin,New Denmark,175500.000000,178833.333333,0.981361


In [15]:
univ

2008q3         2009q2  Price Ratio
State     RegionName                                           
Alabama   Montevallo  127266.666667  125200.000000     1.016507
          Tuscaloosa  139600.000000  136933.333333     1.019474
Alaska    Fairbanks   249966.666667  225833.333333     1.106863
Arizona   Flagstaff   322633.333333  299600.000000     1.076880
          Tempe       228133.333333  207500.000000     1.099438
...                             ...            ...          ...
Wisconsin La Crosse   117300.000000  111933.333333     1.047945
          Madison     194400.000000  191433.333333     1.015497
          Milwaukee   127900.000000  116733.333333     1.095660
          Oshkosh     129966.666667  121200.000000     1.072332
          Waukesha    193066.666667  188200.000000     1.025859

[269 rows x 3 columns]

Now we'll run a ttest comparing the university town values to the non-university towns values and check whether the alternative hypothesis (that the two groups are the same) is true or not as well as the p-value of the confidence.
<br>
If p<0.01 then we reject the null hypothesis, otherwise we cannot reject the null hypothesis.

In [16]:
ttest_ind(mhp['Price Ratio'],univ['Price Ratio'],nan_policy='omit')

Ttest_indResult(statistic=2.7770133719318877, pvalue=0.005496427353633026)

So we can see that pvalue ~ 0.0055 therefore we reject the null hypothesis and can say that:<br>
**University towns in United States have their mean housing prices less effected by recessions.**